# SnapShot preprocessing:
In this notebook, we process the snapshots to reconstruct the travel diaries.

#### adding required packages

In [92]:
import pandas as pd
from scipy.stats import gaussian_kde
import time
import numpy as np
import pickle
import concurrent.futures
from threading import Thread

### specifying the saving location 
## __attention!__ 
later in the `cluster_records` function, we use an address that must be edited from its location, NOT here. 

In [93]:
savingLoc = "Y:/ZahraEftekhar/phase4/"

First, we have to cluster the consecative records that have the same location (i.e., the TAZ).

In [53]:
#we time the process
startTime = time.time()

intervals = [30,60,300,600,900,1200,1500,1800,2100,2400,2700,3000,3300,3600,4500,5400,6300,7200]
for interval in intervals:
    t1= time.time()
    clusterData = {}
    with open("{a}completePLUdata_{b}sec_dict.pickle".format(a=savingLoc,b= interval),'rb') as handle:
        snapDataSplit = pickle.load(handle)
    for i,ID in enumerate(snapDataSplit.keys()):
        if i%11000==0:
            print(interval,": ",(time.time()-t1)//60," min ...")
            t1= time.time()
        person = snapDataSplit[ID]
        person = person.reset_index(drop=False, inplace=False)
        newD = pd.DataFrame()
        for k,v in person.groupby((person.mzr_id.shift() != person.mzr_id).cumsum()):
            newD = newD.append(v.iloc[0,:])
        d = np.array(newD["TIME"].shift(periods=-1, fill_value=v.iloc[-1,:]["TIME"])) - np.array(newD["TIME"])
        d = [pd.Timedelta(d[i]).total_seconds() for i in np.arange(len(d))]
        newD["duration"] = d
        newD.columns = ['start', 'id', 'x', 'y', 'mzr_id', 'duration']
        clusterData[ID] = newD
    with open('{a}clusterData_{b}sec.pickle'.format(a= savingLoc,b = interval), 'wb') as handle:
        pickle.dump(clusterData, handle, protocol=pickle.HIGHEST_PROTOCOL)
print((time.time() - startTime)//60,'minutes')

30 :  6.0  min ...
30 :  7.0  min ...
30 :  7.0  min ...
60 :  2.0  min ...


KeyboardInterrupt: 

In [94]:
def cluster_records(interval,address):
#     savingLoc = "Y:/ZahraEftekhar/phase4/"
    t1= time.time()
    clusterData = {}
    with open("{a}completePLUdata_{b}sec_dict.pickle".format(a=address,b= interval),'rb') as handle:
        snapDataSplit = pickle.load(handle)
    for i,ID in enumerate(snapDataSplit.keys()):
#         if i%11000==0:
#             print(interval,": ",(time.time()-t1)//60," min ...")
#             t1= time.time()
        person = snapDataSplit[ID]
        person = person.reset_index(drop=False, inplace=False)
        newD = pd.DataFrame()
        for k,v in person.groupby((person.mzr_id.shift() != person.mzr_id).cumsum()):
            newD = newD.append(v.iloc[0,:])
        d = np.array(newD["TIME"].shift(periods=-1, fill_value=v.iloc[-1,:]["TIME"])) - np.array(newD["TIME"])
        d = [pd.Timedelta(d[i]).total_seconds() for i in np.arange(len(d))]
        newD["duration"] = d
        newD.columns = ['start', 'id', 'x', 'y', 'mzr_id', 'duration']
        clusterData[ID] = newD
    print(interval, ":  ",(time.time()-t1)//60,"minutes")
    t1= time.time()
    with open('{a}clusterData_{b}sec.pickle'.format(a= address,b = interval), 'wb') as handle:
        pickle.dump(clusterData, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [59]:
intervals = [60,300,600,900]
with concurrent.futures.ThreadPoolExecutor(max_workers=4) as executor:
    executor.submit(cluster_records,intervals)

In [95]:
intervals = [60,300,600,900,1200,1500,1800,2100,2400,2700,3000,3300,3600,4500,5400,6300,7200]
# with concurrent.futures.ProcessPoolExecutor(max_workers=4) as executor:
#     executor.submit(cluster_records,intervals)
num_parallel = 10
num_date = len(intervals)
for i in range(0, num_date, num_parallel):
    
    threads = [Thread(target=cluster_records, args=(interval, savingLoc)) for interval in (intervals[i: i + num_parallel])]
    [th.start() for th in threads]
#     [th.join() for th in threads]
    
